In [1]:
# coding: utf-8

"""
60. KVSの構築
Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．
"""

from redis import StrictRedis
from json import loads
from src.ConfigReader import RedisConfig


def main():
    json_data = []
    with open("./src/artist.json", 'r') as _file:
        for line in _file.readlines():
            # json stringをdic型に変換
            json_data.append(loads(line))

    path = "./src/config.ini"
    redis_config = RedisConfig(path)
    host, port, db = redis_config.read_config()

    r = StrictRedis(host=host, port=port, db=db)

    # 接続しているDBを全消し
    r.flushdb()

    # Redisに登録
    for dic in json_data:
        if "area" in dic:
            r.set(str(dic["id"]) + "_" + dic["name"], dic["area"])


if __name__ == '__main__':
    main()


In [2]:
# coding: utf-8

"""
61. KVSの検索
60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．
"""

from redis import StrictRedis
from sys import argv
from src.ConfigReader import RedisConfig


def main(name):

    path = "./src/config.ini"
    redis_config = RedisConfig(path)
    host, port, db = redis_config.read_config()

    r = StrictRedis(host=host, port=port, db=db)

    # Redisから取得
    artist_name_list = r.keys("*_" + name)

    if artist_name_list:
        for artist_name in artist_name_list:
            _id = artist_name.split("_")[:1][0]
            _name = "".join(artist_name.split("_")[1:])
            print u"ID:{} アーティスト名:{} 活動場所:{}".format(_id, _name, r.get(artist_name))
    else:
        print u"データベースに存在しません"

if __name__ == '__main__':
    
     main("America")


ID:2715 アーティスト名:America 活動場所:United States


In [3]:
# coding: utf-8

"""
62. KVS内の反復処理
60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．
"""

from redis import StrictRedis
from src.ConfigReader import RedisConfig


def main():
    cnt = 0
    path = "./src/config.ini"
    redis_config = RedisConfig(path)
    host, port, db = redis_config.read_config()
    r = StrictRedis(host=host, port=port, db=db)
    for key in r.keys():
        if r.get(key) == "Japan":
            cnt += 1

    print cnt

if __name__ == '__main__':
    main()


22821


In [13]:
# coding: utf-8

"""
63. オブジェクトを値に格納したKVS
KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．
さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．
"""

from sys import argv
from redis import StrictRedis
from json import loads
from src.ConfigReader import RedisConfig


def build_db():
    json_data = []
    with open("src/artist.json", 'r') as _file:
        for line in _file.readlines():
            # json stringをdic型に変換
            json_data.append(loads(line))

    path = "./src/config.ini"
    redis_config = RedisConfig(path)
    host, port, db = redis_config.read_config()
    r = StrictRedis(host=host, port=port, db=1)

    # 接続しているDBを全消し
    r.flushdb()

    # Redisに登録
    for dic in json_data:
        if "tags" in dic:
            for tag in dic["tags"]:

                # ユニークID + アーティスト名, 被タグ数_タグのリストの組み合わせ
                r.sadd(str(dic["id"]) + "_" + dic["name"],
                       str(tag["count"]) + "_" + tag["value"])


def search_db(name):
    r = StrictRedis(host='localhost', port=6379, db=1)

    # Redisから取得
    artist_name_list = r.keys("*_" + name)

    if artist_name_list:
        for artist_name in artist_name_list:
            _id = artist_name.split("_")[:1][0]
            _name = "".join(artist_name.split("_")[1:])

            print u"ID:{} アーティスト名:{}".format(_id, _name)

            for tag in r.smembers(artist_name):
                tag_cnt = tag.split("_")[:1][0]
                tag_name = "".join(tag.split("_")[1:])
                print u"タグ:{} 被タグ数:{}".format(tag_name, tag_cnt)

    else:
        print u"データベースに存在しません"


def main():
    build_db()

if __name__ == '__main__':
    main()


In [17]:
search_db("Cover Dose")


ID:955036 アーティスト名:Cover Dose
タグ:punk 被タグ数:1
タグ:brittany 被タグ数:1
タグ:france 被タグ数:1


In [19]:
# coding: utf-8

"""
64. MongoDBの構築
アーティスト情報（artist.json.gz）をデータベースに登録せよ．
さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value
"""

from json import loads
from pymongo import MongoClient
from src.ConfigReader import MongoDBConfig


def main():

    # jsonファイルの読み込み
    json_data = []
    with open("src/artist.json", 'r') as _file:
        for line in _file.readlines():
            # json stringをdic型に変換
            json_data.append(loads(line))

    path = "./src/config.ini"
    mongodb_config = MongoDBConfig(path)
    host, port = mongodb_config.read_config()
    port = int(port)

    # MongoDBへ登録
    client = MongoClient(host, port)
    db = client.local
    collection = db.artist

    collection.drop()

    # 必要なデータを抽出しながら登録
    for artist in json_data:
        collection.insert_one(artist)

    # インデックスの生成
    collection.create_index("name")
    collection.create_index("aliases")
    collection.create_index("tags")
    collection.create_index("rating")

if __name__ == '__main__':
    main()


In [20]:
# coding: utf-8

"""
65. MongoDBの検索
MongoDBのインタラクティブシェルを用いて，"Queen"というアーティストに関する情報を取得せよ．
さらに，これと同様の処理を行うプログラムを実装せよ．
"""

from pymongo import MongoClient
from src.ConfigReader import MongoDBConfig


def search_artist_info(parm):

    path = "./src/config.ini"
    mongodb_config = MongoDBConfig(path)
    host, port = mongodb_config.read_config()
    port = int(port)

    # MongoDBからパラメータより検索
    client = MongoClient(host, port)
    db = client.local
    collection = db.artist

    return collection.find(parm)


def main(parm):
    artist = search_artist_info(parm)

    for i in artist:
        print i

if __name__ == '__main__':
    main({"name": "Queen"})


{u'name': u'Queen', u'area': u'Japan', u'gender': u'Female', u'tags': [{u'count': 1, u'value': u'kamen rider w'}, {u'count': 1, u'value': u'related-akb48'}], u'sort_name': u'Queen', u'ended': True, u'gid': u'420ca290-76c5-41af-999e-564d7c71f1a7', u'_id': ObjectId('5ad75ae13a8eb0129553d8b6'), u'type': u'Character', u'id': 701492, u'aliases': [{u'name': u'Queen', u'sort_name': u'Queen'}]}
{u'rating': {u'count': 24, u'value': 92}, u'begin': {u'date': 27, u'month': 6, u'year': 1970}, u'name': u'Queen', u'area': u'United Kingdom', u'tags': [{u'count': 2, u'value': u'hard rock'}, {u'count': 1, u'value': u'70s'}, {u'count': 1, u'value': u'queen family'}, {u'count': 1, u'value': u'90s'}, {u'count': 1, u'value': u'80s'}, {u'count': 1, u'value': u'glam rock'}, {u'count': 4, u'value': u'british'}, {u'count': 1, u'value': u'english'}, {u'count': 2, u'value': u'uk'}, {u'count': 1, u'value': u'pop/rock'}, {u'count': 1, u'value': u'pop-rock'}, {u'count': 1, u'value': u'britannique'}, {u'count': 1, u'

66. インタラクティブシェルで実行のため省略

In [21]:
# coding: utf-8

"""
67. 複数のドキュメントの取得
特定の（指定した）別名を持つアーティストを検索せよ．
"""

from sys import argv
from pymongo import MongoClient
from src.ConfigReader import MongoDBConfig


def search_artist_name(parm):

    query = parm
    query["aliases"] = {"$exists": "false"}

    path = "./src/config.ini"
    mongodb_config = MongoDBConfig(path)
    host, port = mongodb_config.read_config()
    port = int(port)

    # MongoDBからパラメータより検索
    client = MongoClient(host, port)
    db = client.local
    collection = db.artist

    return collection.find(query)


def main(parm):
    artist = search_artist_name(parm)

    for i in artist:
        print i

if __name__ == '__main__':
    main({"name": "Queen"})


{u'name': u'Queen', u'area': u'Japan', u'gender': u'Female', u'tags': [{u'count': 1, u'value': u'kamen rider w'}, {u'count': 1, u'value': u'related-akb48'}], u'sort_name': u'Queen', u'ended': True, u'gid': u'420ca290-76c5-41af-999e-564d7c71f1a7', u'_id': ObjectId('5ad75ae13a8eb0129553d8b6'), u'type': u'Character', u'id': 701492, u'aliases': [{u'name': u'Queen', u'sort_name': u'Queen'}]}
{u'rating': {u'count': 24, u'value': 92}, u'begin': {u'date': 27, u'month': 6, u'year': 1970}, u'name': u'Queen', u'area': u'United Kingdom', u'tags': [{u'count': 2, u'value': u'hard rock'}, {u'count': 1, u'value': u'70s'}, {u'count': 1, u'value': u'queen family'}, {u'count': 1, u'value': u'90s'}, {u'count': 1, u'value': u'80s'}, {u'count': 1, u'value': u'glam rock'}, {u'count': 4, u'value': u'british'}, {u'count': 1, u'value': u'english'}, {u'count': 2, u'value': u'uk'}, {u'count': 1, u'value': u'pop/rock'}, {u'count': 1, u'value': u'pop-rock'}, {u'count': 1, u'value': u'britannique'}, {u'count': 1, u'

In [23]:
# coding: utf-8

"""
68. ソート
"dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティスト・トップ10を求めよ．
"""

from pymongo import MongoClient
from src.ConfigReader import MongoDBConfig


def search_top_artist():

    path = "./src/config.ini"
    mongodb_config = MongoDBConfig(path)
    host, port = mongodb_config.read_config()
    port = int(port)

    # MongoDBからパラメータより検索
    client = MongoClient(host, port)
    db = client.local
    collection = db.artist

    # 問い合わせ内容
    query = {"tags": {"$elemMatch": {"value": "dance"}}}
    order_key = "rating.count"
    order_value = -1
    top = 10

    return collection.find(query).sort(order_key, order_value).limit(top)


def main():
    artist = search_top_artist()

    for i in artist:
        print i

if __name__ == '__main__':
    main()


{u'rating': {u'count': 26, u'value': 88}, u'begin': {u'date': 16, u'month': 8, u'year': 1958}, u'name': u'Madonna', u'tags': [{u'count': 1, u'value': u'dance-pop'}, {u'count': 1, u'value': u'electropop'}, {u'count': 1, u'value': u'tell me'}, {u'count': 1, u'value': u'pop and chart'}, {u'count': 1, u'value': u'multiple ipi'}, {u'count': 1, u'value': u'electronic'}, {u'count': 1, u'value': u'am\xe9ricain'}, {u'count': 1, u'value': u'usa'}, {u'count': 1, u'value': u'singer'}, {u'count': 1, u'value': u'chanteur'}, {u'count': 1, u'value': u'american'}, {u'count': 4, u'value': u'pop'}, {u'count': 1, u'value': u'greatest hits'}, {u'count': 1, u'value': u'dance'}], u'gender': u'Female', u'area': u'United States', u'sort_name': u'Madonna', u'ended': True, u'gid': u'79239441-bfd5-4981-a70c-55c3f15c1287', u'_id': ObjectId('5ad75b4c3a8eb0129555b2b3'), u'type': u'Person', u'id': 89}
{u'rating': {u'count': 23, u'value': 84}, u'begin': {u'date': 21, u'month': 11, u'year': 1965}, u'name': u'Bj\xf6rk',